In [1]:
import dash
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
import numpy as np

In [2]:
# Initialisation de l'application
app = dash.Dash(__name__)

# Données
data = {
    "Nom": ["Lycée Babacar Cobar Ndao", "Cem Waly Thiobane", "École élémentaire Waly Fatha Ndiaye",
            "École Élémentaire Diamaguene TP", "Lycée Diamaguene TP", "Elhadji Ndéné Diodio Ndiaye",
            "CEM 2 Babacar Cobar Ndao", "CEM 1 Babacar Cobar Ndao", "École El hadji Fana Cissé",
            "Lycée Franco-Arabe", "École Notre Dame", "École Kaffrine 6", "École élémentaire Kaffrine 10",
            "École élémentaire Kaffrine 3", "École élémentaire Kaffrine 9", "École élémentaire Tagouthie Waly Ndao",
            "École élémentaire Kaffrine 8", "École élémentaire Kaffrine 12"],
    "Niveau": ["Lycée", "Collège", "Élémentaire", "Élémentaire", "Lycée", "Élémentaire", "Collège", "Collège", "Élémentaire", "Lycée", "Élémentaire", "Élémentaire", "Élémentaire", "Élémentaire", "Élémentaire", "Élémentaire", "Élémentaire", "Élémentaire"],
    "Effectif": [1937, 906, 644, 653, 1500, 635, 874, 853, 389, 858, 624, 658, 305, 536, 563, 372, 418, 341],
    "Garçons": [771, 382, 286, 271, 733, 256, 403, 364, 194, 401, 329, 349, 142, 265, 257, 192, 198, 176],
    "Filles": [1166, 524, 358, 382, 767, 379, 471, 488, 195, 515, 295, 309, 163, 271, 306, 180, 220, 165],
    "Nombre de Professeurs": [48, 23, 16, 16, 34, 15, 23, 22, 9, 25, 18, 15, 10, 14, 14, 14, 13, 12],
    "Salles de classes": [31, 14, 12, 15, 18, 13, 18, 17, 11, 10, 13, 13, 8, 12, 13, 13, 12, 10],
    "Classes fonctionnelles": [31, 14, 12, 12, 18, 13, 18, 16, 11, 10, 13, 13, 8, 12, 12, 12, 12, 10],
    "Classes non fonctionnelles": [0, 0, 0, 3, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0],
}

df = pd.DataFrame(data)

In [3]:
# Calcul du nombre moyen d'élèves par classe fonctionnelle
df["Élèves par classe fonctionnelle"] = df["Effectif"] / df["Classes fonctionnelles"]

# Layout de l'application
app.layout = html.Div([
    html.H1("Tableau de bord des écoles de Kaffrine"),
    html.H2("Analyse de la répartition et des caractéristiques des écoles"),
    dcc.Dropdown(
        id='niveau-dropdown',
        options=[{'label': niveau, 'value': niveau} for niveau in df['Niveau'].unique()],
        value='Lycée'
    ),
    dcc.Graph(id='students-chart'),
    dcc.Graph(id='gender-chart'),
    dcc.Graph(id='teachers-chart'),
    dcc.Graph(id='classes-chart'),
    dcc.Graph(id='students-per-class-chart'),
    dash_table.DataTable(
        id='data-table',
        columns=[{"name": i, "id": i} for i in df.columns],
        data=df.to_dict('records'),
        page_size=10,
    ),
    html.Button("Télécharger les données CSV", id="btn-download"),
]) 

In [4]:
# Callback pour mettre à jour les graphiques
@app.callback(
    [Output('students-chart', 'figure'),
     Output('gender-chart', 'figure'),
     Output('teachers-chart', 'figure'),
     Output('classes-chart', 'figure'),
     Output('students-per-class-chart', 'figure')],
    [Input('niveau-dropdown', 'value')]
)
def update_charts(selected_niveau):
    filtered_df = df[df['Niveau'] == selected_niveau]

    # Vérifier si les données existent après le filtrage
    if filtered_df.empty:
        students_chart = px.bar(title="Pas de données disponibles")
        gender_chart = px.bar(title="Pas de données disponibles")
        teachers_chart = px.bar(title="Pas de données disponibles")
        classes_chart = px.bar(title="Pas de données disponibles")
        students_per_class_chart = px.bar(title="Pas de données disponibles")
    else:
        students_chart = px.bar(filtered_df, x='Nom', y='Effectif', title="Nombre d'élèves par école")
        gender_chart = px.bar(filtered_df, x='Nom', y=['Garçons', 'Filles'], title="Répartition des genres par école")
        teachers_chart = px.bar(filtered_df, x='Nom', y='Nombre de Professeurs', title="Nombre de professeurs par école")
        classes_chart = px.bar(filtered_df, x='Nom', y=['Classes fonctionnelles', 'Classes non fonctionnelles'], title="Nombre de classes fonctionnelles et non fonctionnelles")
        students_per_class_chart = px.bar(filtered_df, x='Nom', y='Élèves par classe fonctionnelle', title="Nombre d'élèves par classe fonctionnelle")

    return students_chart, gender_chart, teachers_chart, classes_chart, students_per_class_chart

In [5]:
# Callback pour le téléchargement CSV
@app.callback(
    Output("download", "data"),
    [Input("btn-download", "n_clicks")],
    prevent_initial_call=True,
)
def download_csv(n_clicks):
    return send_data_frame(df.to_csv, "data_kaffrine.csv")

In [6]:
#Démarrer l'application dans Jupyter avec un port différent
app.run_server(mode='inline', port=8053)

In [7]:
!pip freeze > requirements.txt